In [1]:
import sys
sys.path.append("../")
# import cupy, cudf

In [2]:
import gc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from skopt import BayesSearchCV
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from tqdm import tqdm

In [3]:
from utils.eval_helpers import plot_roc_curves, plot_feature_importance, amex_metric
from utils.eda_helpers import plot_missing_proportion_barchart

In [4]:
RAW_DATA_PATH = "../raw_data"
os.listdir(RAW_DATA_PATH)

['train_labels.csv',
 '.DS_Store',
 'train_data.parquet',
 'test_data.parquet',
 'test_data.ftr',
 'train_data.csv',
 'train_data.ftr',
 'test_data.csv',
 'sample_submission.csv']

In [5]:
PROCESSED_DATA_PATH = "../processed_data"
SUBMISSION_DATA_PATH = "../submissions"
CATEGORY_COLUMNS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [6]:
%load_ext autoreload
%autoreload

### Preprocessing Train

In [7]:
# train_data = pd.read_parquet(f"{RAW_DATA_PATH}/train_data.parquet")
train_data = pd.read_feather(f"{RAW_DATA_PATH}/train_data.ftr")
train_data["S_2"] = pd.to_datetime(train_data["S_2"])

In [8]:
train_data.shape

(5531451, 190)

In [9]:
train_data.columns

Index(['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41',
       'B_3',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=190)

In [10]:
train_labels = pd.read_csv(f"{RAW_DATA_PATH}/train_labels.csv")

In [11]:
train_labels.shape

(458913, 2)

In [12]:
train_labels.columns

Index(['customer_ID', 'target'], dtype='object')

In [13]:
train_data = train_data.sort_values(by=["customer_ID", "S_2"])

In [14]:
train_data = train_data.merge(train_labels, on="customer_ID", how="left")

### Preparation

In [15]:
def get_cols(df, key):
    return [col for col in df.columns if key in col]

In [16]:
special_columns = ['customer_ID', 'S_2', 'target'] + get_cols(train_data, "has") + get_cols(train_data, "sign")

In [17]:
all_cols = [c for c in train_data.columns if c not in special_columns]
cat_features = CATEGORY_COLUMNS
num_features = [col for col in all_cols if col not in CATEGORY_COLUMNS]

In [18]:
len(all_cols), len(cat_features), len(num_features)

(188, 11, 177)

### Aggregation (Train)

In [19]:
def get_agg_summary(original_df):
    agg_summary = original_df.groupby("customer_ID").agg(num_records=("S_2", "count"), 
                                                         max_date=("S_2", "max"),
                                                         min_date=("S_2", "min"))
    agg_summary["days"] = (agg_summary["max_date"] - agg_summary["min_date"]).dt.days
    agg_summary["record_per_day"] = agg_summary["days"] / agg_summary["num_records"]
    return agg_summary

In [20]:
train_agg_summary = get_agg_summary(train_data)

In [21]:
train_data_agg = train_data.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last', 'first'])
train_data_agg.columns = ['_'.join(x) for x in train_data_agg.columns]

In [22]:
train_cat_agg = train_data.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]

In [23]:
train_agg = pd.concat([train_agg_summary, train_data_agg, train_cat_agg], axis=1)
del train_agg_summary, train_data_agg, train_cat_agg

In [26]:
train_agg = train_agg.reset_index()

In [27]:
train_agg.shape

(458913, 1101)

In [28]:
train_agg.head()

,customer_ID,num_records,max_date,min_date,days,record_per_day,P_2_mean,P_2_std,P_2_min,P_2_max,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,13,2018-03-13,2017-03-09,369,28.384615,0.933594,0.024194,0.868652,0.960449,...,1,13,O,1,0,NaN,0,13,6.0,1
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,13,2018-03-25,2017-03-01,389,29.923077,0.899902,0.022097,0.861328,0.929199,...,1,13,O,1,0,NaN,0,13,6.0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,13,2018-03-12,2017-03-11,366,28.153846,0.878418,0.028837,0.797852,0.904297,...,1,13,R,1,0,NaN,0,13,6.0,1
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,13,2018-03-29,2017-03-31,363,27.923077,0.599121,0.020082,0.567383,0.623535,...,1,13,O,1,0,NaN,0,13,3.0,3
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,13,2018-03-30,2017-03-30,365,28.076923,0.891602,0.042316,0.805176,0.940430,...,1,13,O,1,13,1.0,1,13,6.0,1


In [29]:
# train_agg.to_parquet(f"{PROCESSED_DATA_PATH}/train_agg_data.parquet")
train_agg.to_feather(f"{PROCESSED_DATA_PATH}/train_agg_data.ftr")

In [ ]:
# train_agg_ = pd.read_parquet(f"{PROCESSED_DATA_PATH}/train_agg_data.parquet")

In [30]:
del train_agg

### Aggregation (Test)

In [31]:
# test_data = pd.read_parquet(f"{RAW_DATA_PATH}/test_data.parquet")
test_data = pd.read_feather(f"{RAW_DATA_PATH}/test_data.ftr")
test_data["S_2"] = pd.to_datetime(test_data["S_2"])

In [32]:
special_columns = ['customer_ID', 'S_2', 'target']

In [33]:
all_cols = [c for c in test_data.columns if c not in special_columns]
cat_features = CATEGORY_COLUMNS
num_features = [col for col in all_cols if col not in CATEGORY_COLUMNS]

In [34]:
len(all_cols), len(cat_features), len(num_features)

(188, 11, 177)

In [35]:
test_data.shape

(11363762, 190)

In [36]:
test_data = test_data.sort_values(by=["customer_ID", "S_2"])

In [37]:
test_agg_summary = get_agg_summary(test_data)

In [38]:
test_data_agg = test_data.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min'])
test_data_agg.columns = ['_'.join(x) for x in test_data_agg.columns]

In [39]:
test_agg = pd.concat([test_agg_summary, test_data_agg], axis=1)
del test_agg_summary, test_data_agg

In [40]:
test_data_agg = test_data.groupby("customer_ID")[num_features].agg(['max', 'last', 'first'])
test_data_agg.columns = ['_'.join(x) for x in test_data_agg.columns]

In [41]:
test_cat_agg = test_data.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

In [42]:
test_agg = pd.concat([test_agg, test_data_agg, test_cat_agg], axis=1)
del test_data_agg, test_cat_agg

In [43]:
test_agg = test_agg.reset_index()

In [44]:
test_agg.shape

(924621, 1101)

In [45]:
test_agg.head()

,customer_ID,num_records,max_date,min_date,days,record_per_day,P_2_mean,P_2_std,P_2_min,D_39_mean,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,9,2019-10-12,2019-02-19,235,26.111111,0.601562,0.020198,0.568848,0.069763,...,1,5,U,1,0,NaN,0,5,6.0,1
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,13,2019-04-15,2018-04-22,358,27.538462,0.862305,0.031432,0.794434,0.154297,...,1,13,O,1,0,NaN,0,13,6.0,1
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,13,2019-10-16,2018-10-03,378,29.076923,0.749023,0.061435,0.673340,0.181885,...,1,13,U,2,13,1.0,1,13,4.0,2
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,13,2019-04-22,2018-04-05,382,29.384615,0.474609,0.028883,0.428467,0.470459,...,1,13,R,1,0,NaN,0,13,5.0,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,13,2019-10-22,2018-10-17,370,28.461538,0.323975,0.049886,0.254395,0.353027,...,1,13,R,2,0,NaN,0,13,5.0,2


In [46]:
# test_agg.to_parquet(f"{PROCESSED_DATA_PATH}/test_agg_data.parquet")
test_agg.to_feather(f"{PROCESSED_DATA_PATH}/test_agg_data.ftr")

In [47]:
del test_agg

In [48]:
# test_agg_ = pd.read_parquet(f"{PROCESSED_DATA_PATH}/test_agg_data.parquet")

#### EDA & Data Cleaning

#### Missing value analysis (Pre Simple Imputation)

In [ ]:
missing_prop_df = plot_missing_proportion_barchart(train_data)

#### Simple Imputation

In [ ]:
def create_has_col(df, col):
    has_col = f"has_{col}"
    df.loc[:, has_col] = 0
    df.loc[~df[col].isnull(), has_col] = 1
    return df

In [ ]:
def create_sign_col(df, col):
    sign_col = f"{col}_sign"
    df[sign_col] = df[col].apply(lambda x: 0 if x == 0 else x / abs(x))
    return df

In [ ]:
def apply_all_fillna(df):
    # Simple Fill NA with 0
    for col in ["D_87", "D_88", "B_39", "B_42"]:
        df[col] = df[col].fillna(0)  # .apply(lambda x: (abs(x) + x) / 2).fillna(0)
    # Create has column
    for col in ["D_110", "D_111", "D_132", "D_134", "D_135", "D_136", "D_137", "D_138"]:
        df = create_has_col(df, col=col)
    # Create sign column
    for col in ["B_39"]:
        df = create_sign_col(df, col=col)
    return df

In [ ]:
# ! Fill NA
train = apply_all_fillna(train)
test = apply_all_fillna(test)

In [ ]:
train["D_102"].min(), train["D_102"].mean(), train["D_102"].max()

In [ ]:
train["D_102"].astype("float32").describe()

In [ ]:
train.loc[train["D_102"].isnull()]["D_102"]

In [ ]:
train["D_102"].describe()

In [ ]:
train.loc[train["D_102"].astype(str) == "inf"]

#### Missing value analysis (Post Simple Imputation)

In [ ]:
missing_prop_df = plot_missing_proportion_barchart(train)

In [ ]:
array = train_data.loc[train_data["customer_ID"] == train_data["customer_ID"][540]]["P_2"].astype("float32")

In [ ]:
np.polyfit(x=range(len(array)), y=array, deg=1)

In [ ]:
def calc_grad(array):
    if len(array) >= 2:
        gradient, y_intercept = np.polyfit(x=range(len(array)), y=array.astype("float32"), deg=1)
        return gradient
    else:
        return 0

In [ ]:
def calc_intercept(array):
    if len(array) >= 2:
        gradient, y_intercept = np.polyfit(x=range(len(array)), y=array.astype("float32"), deg=1)
        return y_intercept
    else:
        return array.values[0]

In [ ]:
temp = train.groupby("customer_ID").agg(P_2_grad=("P_2", calc_grad),
                                        P_2_intercept=("P_2", calc_intercept)).reset_index()
train_agg_summary = train_agg_summary.merge(temp, on="customer_ID", how="left")

In [ ]:
temp = test.groupby("customer_ID").agg(P_2_grad=("P_2", calc_grad),
                                       P_2_intercept=("P_2", calc_intercept)).reset_index()
test_agg_summary = test_agg_summary.merge(temp, on="customer_ID", how="left")

### Has_xx col Agg

In [ ]:
train_has_col_mean_df = train[["customer_ID"] + get_cols(train, "has") + get_cols(train, "sign")].groupby("customer_ID").agg(["mean"])
test_has_col_mean_df = test[["customer_ID"] + get_cols(test, "has") + get_cols(test, "sign")].groupby("customer_ID").agg(["mean"])

In [ ]:
train_has_col_mean_df.columns = ['_'.join(x) for x in train_has_col_mean_df.columns]
test_has_col_mean_df.columns = ['_'.join(x) for x in test_has_col_mean_df.columns]

In [ ]:
train_agg = train_agg.merge(train_has_col_mean_df, on="customer_ID")
test_agg = test_agg.merge(test_has_col_mean_df, on="customer_ID")

In [ ]:
train_agg.to_feather(f"{PROCESSED_DATA_PATH}/train_agg_data.ftr")

In [ ]:
test_agg.to_feather(f"{PROCESSED_DATA_PATH}/test_agg_data.ftr")

In [ ]:
train_cols_unique_count = train_data.nunique()

In [ ]:
less_unique_columns = train_cols_unique_count[train_cols_unique_count <= 300].index.tolist()
print(less_unique_columns)

In [ ]:
for column in less_unique_columns:
    print(f"Column {column}")
    print(train_data[column].unique(), end="\n\n")

In [ ]:
def set_category_columns(df):
    pass

In [ ]:
train_data.loc[:, cat_features].

### Train Val Split

In [ ]:
train_labels.insert(0, "cid", train_labels['customer_ID'].apply(hash).astype('int64'))
train_labels.head(5)

In [ ]:
train_labels = train_labels.sort_values(by="cid").reset_index(drop=True)

In [ ]:
train_ = train_agg.merge(train_labels[["cid", "target"]], on="cid", how="left")

In [ ]:
train_["dummy"] = np.random.randn(train_.shape[0])

In [ ]:
train, val = train_test_split(train_, test_size=0.2, random_state=1020, stratify=train_["target"])

In [ ]:
train["target"].mean(), val["target"].mean()

In [ ]:
def process_and_feature_engineer(df):
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    num_features = [col for col in all_cols if col not in cat_features]

    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    
    test_num_agg2 = df.groupby("customer_ID")[num_features].agg(['first', 'last'])
    test_num_agg2.columns = ['_'.join(x) for x in test_num_agg2.columns]

    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

    df = pd.concat([test_num_agg, test_num_agg2, test_cat_agg], axis=1).reset_index()
    print(df.columns)
    del test_num_agg, test_num_agg2, test_cat_agg
    print('shape after engineering', df.shape )
    
    min_features = [col for col in df.columns if col.endswith("_min")]
    max_features = [col for col in df.columns if col.endswith("_max")]
    for mini, maxi in zip(min_features, max_features):
        df.loc[:, maxi.replace("_max", "_range")] = df[maxi] - df[mini]
    
    first_features = [col for col in df.columns if col.endswith("_first")]
    last_features = [col for col in df.columns if col.endswith("_last")]
    for first, last in zip(first_features, last_features):
        df.loc[:, first.replace("_first", "_disp")] = df[last] - df[first]
        
    mean_features = [col for col in df.columns if col.endswith("_mean")]
    std_features = [col for col in df.columns if col.endswith("_std")]
    for mean, std in zip(mean_features, std_features):
        # CV stands for dispersion ratio => Coefficient of Variation
        df.loc[:, mean.replace("_mean", "_cv")] = df[std] / df[mean]
    
    range_features = [col for col in df.columns if col.endswith("_range")]
    for range_, std in zip(range_features, std_features):
        # DR stands for dispersion ratio => Standard Deviation / Range
        df.loc[:, range_.replace("_range", "_dr")] = df[std] / df[range_]
    return df